In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path
import pickle

In [ ]:
upstream = ['bespoke-eda', 'get-data']
product = None

In [ ]:
X = pickle.loads(Path(upstream['bespoke-eda']['X']).read_bytes())
categorical_columns = pickle.loads(Path(upstream['bespoke-eda']['categorical_columns']).read_bytes())
y = pickle.loads(Path(upstream['bespoke-eda']['y']).read_bytes())
train = pickle.loads(Path(upstream['get-data']['train']).read_bytes())

## Data Preparation

In [30]:
## One hot encode the categorical features
##Leave all categories to represent missing values 
X_oh_encoded = pd.concat([X, pd.get_dummies(X[categorical_columns])], axis=1)
X_oh_encoded = X_oh_encoded.drop(columns=categorical_columns)

## fill the missing numerical columns
X_oh_encoded['LoanAmount'].fillna(X_oh_encoded['LoanAmount'].mean(), inplace=True)
X_oh_encoded['Loan_Amount_Term'].fillna(X_oh_encoded['Loan_Amount_Term'].median(), inplace=True)
X_oh_encoded['Credit_History'].fillna(X_oh_encoded['Credit_History'].mode()[0], inplace=True)
X_oh_encoded.isna().sum()

ApplicantIncome            0
CoapplicantIncome          0
LoanAmount                 0
Loan_Amount_Term           0
Credit_History             0
Gender_Female              0
Gender_Male                0
Married_No                 0
Married_Yes                0
Dependents_0               0
Dependents_1               0
Dependents_2               0
Dependents_3+              0
Education_Graduate         0
Education_Not Graduate     0
Self_Employed_No           0
Self_Employed_Yes          0
Property_Area_Rural        0
Property_Area_Semiurban    0
Property_Area_Urban        0
dtype: int64

In [31]:
X_train, X_val, y_train, y_val = train_test_split(train.drop(columns=['Loan_ID', 'Loan_Status']), train['Loan_Status'])

In [32]:
X_train, X_val, y_train, y_val = train_test_split(X_oh_encoded, y)

In [ ]:
Path(product['X_train']).parent.mkdir(exist_ok=True, parents=True)
Path(product['X_train']).write_bytes(pickle.dumps(X_train))

Path(product['X_val']).parent.mkdir(exist_ok=True, parents=True)
Path(product['X_val']).write_bytes(pickle.dumps(X_val))

Path(product['y_train']).parent.mkdir(exist_ok=True, parents=True)
Path(product['y_train']).write_bytes(pickle.dumps(y_train))

Path(product['y_val']).parent.mkdir(exist_ok=True, parents=True)
Path(product['y_val']).write_bytes(pickle.dumps(y_val))